In [92]:
import pandas as pd
import numpy as np
import os
import ast
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error


   




#These are features we'll exclude, as they're not useful for generalization, and they're specific to the data set used in summer 2019 developing
non_model_features = ['start','submission_time','end','device_id','country','developer_code', 
                      'village_code', 'joint_code', 'gps_coordinates',
                       'gps_coordinates_latitude', 'gps_coordinates_longitude',
                       'gps_coordinates_altitude', 'gps_coordinates_precision',
                       'meter_number','survey_version','meta_instanceid', 'id', 'uuid', 'index', 
                       'parent_index', 'customer_id','start_date','end_date','tariff_min','tariff_max']


def evaluate_regression(predictions, labels):
    errors = abs(predictions - labels)
    print('MAE', mean_absolute_error(predictions, labels))
    print('MSE', mean_squared_error(predictions, labels))
    print('R2', r2_score(labels, predictions))
    print('Average Error: ', np.mean(errors), 'kwh')

def cap_outlier(df, col, max_threshold):
    '''
    Caps the specified column at the max_threshold 
    '''
    df.loc[df[col]>max_threshold, col] = max_threshold
    return df

def train_test_split(df, test_pct, random_state=1):
    '''
    My own train_test_split helper function - keeps target in same DF
    '''
    model_df_shuf = df.sample(frac=1, random_state=random_state).reset_index(drop=True)

    #Data is shuffled so we can just take bottom 10% for test set
    test_pct = 0.1
    test_index = int(model_df_shuf.shape[0]) * test_pct
    test_df = model_df_shuf.loc[:test_index]
    train_df = model_df_shuf.loc[test_index:]
    
    return train_df, test_df


class FeatureImportanceSummary(object):
    '''
    Takes in a data frame, features to drop, target_variable
    
        Builds a RF to compute feature importance scores, also does linear regression to get coefficients and signs
    
        Produces a dataFrame with the feature importance summary
        Gives indices within the dataframe at different thresholds of feature importance
    '''
    def __init__(self, df, non_model_features, Y):    
        self.df = df
        self.non_model_features = non_model_features #Features that shouldn't be in a model
        self.Y = Y #Target variables
        self.feat_imp_df = None
        self.feat_imp_thresholds = [0.25, 0.5, 0.8, 0.9, 0.95, 0.99] #thresholds for normalized feature importance
        self.feat_imp_index = {}

    def get_feature_importance_summary(self):

        model_train = self.df.drop(non_model_features, axis=1).copy()

        #Fit RF model to get its feature importances
        rf = RandomForestRegressor(n_estimators=500, max_depth=10, min_samples_split=5)
        rf.fit(model_train.drop(self.Y,axis=1), model_train[self.Y])

        #Add to DF
        self.feat_imp_df = pd.DataFrame({'x':model_train.drop(self.Y,axis=1).columns.values,'imp':rf.feature_importances_})
        self.feat_imp_df = self.feat_imp_df.sort_values(by=['imp'], ascending=False).reset_index(drop=True)

        #Get sign of relationship from univariate linear regressions
        xs = self.feat_imp_df.x.values

        betas = []
        signs = []

        for x in xs:
            beta = LinearRegression().fit(model_df[x].values.reshape(-1,1), model_df[self.Y]).coef_[0]
            betas.append(beta)
            if beta < 0:
                signs.append('neg')
            else:
                signs.append('pos')

        self.feat_imp_df['lin_coef'] = betas
        self.feat_imp_df['direction'] = signs
        
        self._get_feature_imp_cutoffs()


    def _get_feature_imp_cutoffs(self):

        cs = np.cumsum(self.feat_imp_df.imp.values)
        
        for cut in self.feat_imp_thresholds:
            self.feat_imp_index[cut] = np.argmin((cs - cut)**2)
            

class ModelCVWrapper(object):
    
    def __init__(self, n_splits):
        self.cv_summary_df = None
        self.model_type = None
        self.n_splits = n_splits
        
        
    def _get_default_grid(self):
        
        if self.model_type == 'classification':
            rf_grid = {'n_estimators':[200,500], 'max_depth':[5,10], 'criterion':['entropy']}
            gbdt_grid = {'n_estimators':[100,200,500],'max_depth':[5,7], 
                    'learning_rate':[0.05,0.1]}
            grid_dict = {RandomForestClassifier():rf_grid, GradientBoostingClassifier(): gbdt_grid}            
        else:
            rf_grid = {'n_estimators':[200,500], 'max_depth':[5,10], 'criterion':['mse','mae']}
            gbdt_grid = {'n_estimators':[100,200,500],'max_depth':[5,7], 'loss':['ls'], 'alpha':[0.5],
                    'learning_rate':[0.05,0.1]}
            grid_dict = {RandomForestRegressor():rf_grid, GradientBoostingRegressor(): gbdt_grid}
            
        return grid_dict
        
    def get_model_cv(self, train_df, test_df, Y, feature_importance, thresholds,
                          grid_dict=None, print_status=True):
        '''
        Inputs:
        train_df is the training data, CV will be applied to it
        test_df is the out of sample 
        grid_dict has the format of : {sklearn.model_type: parameter_grid}
        feature_importance is a FeatureImportanceSummary object
        thresholds -> the feature importance thresholds you want to test. 

        Performs cross-validation for each model type, on different subsets of model features
        
        Automatically determines regression or classification based on target value
    
        Returns a dataframe of cross-validation results
        '''
        
        #Determine if Y is a continuous or discrete variable
        num_distinct_targets = len(model_df['avg_consumption'].value_counts())
            
        if num_distinct_targets <= 10:
            self.model_type = 'classification'
        else:
            self.model_type = 'regression'
        
        if grid_dict is None:
            grid_dict = self._get_default_grid()


        kf = KFold(n_splits=self.n_splits, random_state=10, shuffle=False)

        #Get the index and create a list of features at a certain feat imp threshold
        ordered_x = feature_importance.feat_imp_df.x.values
        
        subsets = []
        for thresh in thresholds:
            subsets.append(ordered_x[0:feature_importance.feat_imp_index[thresh]])
                
    
        results = []
        for i, subset in enumerate(subsets):
            for model in grid_dict:
                
                if print_status:
                    print('Running Treshold {}'.format(thresholds[i]))
                
                
                #Run a grid search CV on the particular classifier
                mod = GridSearchCV(model, grid_dict[model], cv=kf, scoring='neg_mean_squared_error')
                mod.fit(train_df[subset], train_df[Y])
        
                #Get Mean-Squared Error on ttest set
                test_err = ((mod.best_estimator_.predict(test_df[subset]) - test_df[Y])**2).mean()
            
                #Append results to a DF
                results.append((type(model).__name__, len(subset), mod.best_score_, str(mod.best_params_), test_err))

        cv_summary_df = pd.DataFrame(results, columns=['algo_string','subset_index','cv_score','params','test_score'])
        cv_summary_df = cv_summary_df.sort_values(by=['cv_score'], ascending=False).reset_index(drop=True)
        
        return cv_summary_df

        
    def increment_cv(self, train_df, test_df, Y, feature_importance, thresholds,
                       grid_dict=None):
        '''
        Adds more test cases to cv - need to think about this more
        '''
        new_summary_df = self.get_model_cv(train_df, test_df, Y, feature_importance, thresholds)

        self.cv_summary_df = pd.concat([self.cv_summary_df, new_summary_df])
        self.cv_summary_df = self.cv_summary_df.sort_values(by=['cv_score'], ascending=False).reset_index(drop=True)
        
        self.get_best_model(train_df, test_df, Y, feature_importance)

        
    def get_best_model(self, train_df, test_df, Y, feature_importance, thresholds = [0.8, 0.9, 0.95, 0.99],
                       grid_dict=None):
        '''
        Fit a model to the best cv results
        '''
        
        if self.cv_summary_df is None:
            self.cv_summary_df = self.get_model_cv(train_df, test_df, Y, feature_importance, thresholds, grid_dict)
    
        #Get best model
        best_params = ast.literal_eval(self.cv_summary_df.loc[0]['params'])

        best_subset_index = self.cv_summary_df.loc[0]['subset_index']
        self.best_subset = feature_importance.feat_imp_df.x.values[0:best_subset_index]
        
        #This converts the class string back a class, and uses the best parameters found in CV
        self.best_model = eval(self.cv_summary_df.loc[0]['algo_string'])(**best_params) 
        self.best_model.fit(train_df[self.best_subset], train_df[Y])


## Test Code

Read in data

In [ ]:
data_dir_june = '/Users/briandalessandro/Documents/CrossBoundary/E4I-Datasets/June_2019_DataShare/'
model_df = pd.read_csv(data_dir_june + 'training_all_in.csv')


Get a ranked list of features by their feature importance. Use the convenience class created for this

In [52]:
fi = FeatureImportanceSummary(model_df, non_model_features, 'avg_consumption')
fi.get_feature_importance_summary()

Take a quick look at the top K features that contain 80% of the normalized information gain (the importance metric)

In [45]:
fi.feat_imp_df.loc[0:fi.feat_imp_index[0.8]].head()

,x,imp,lin_coef,direction
0,tariff,0.400476,-0.094498,neg
1,non_self_generated_electricity_monthly_consump...,0.055983,0.000003,pos
2,years_in_community,0.048476,-0.002154,neg
3,energy,0.033691,0.004043,pos
4,phone,0.020734,0.003265,pos


Split the data

In [44]:
train_df, test_df = train_test_split(model_df, 0.1)

### Regression

Get a best performing regression model using grid search and cross-validation (this can take some time)

In [97]:
reg_cv = ModelCVWrapper(5)

rf_grid = {'n_estimators':[100,200], 'max_depth':[5,10], 'criterion':['mse']}
grid_dict = {RandomForestRegressor():rf_grid}


reg_cv.get_best_model(train_df, test_df, 'avg_consumption', fi, [0.8,0.9], grid_dict)

Running Treshold 0.8
Running Treshold 0.9


Look at results from the test

In [98]:
reg_cv.cv_summary_df

,algo_string,subset_index,cv_score,params,test_score
0,RandomForestRegressor,26,-0.042120,"{'criterion': 'mse', 'max_depth': 5, 'n_estima...",0.015160
1,RandomForestRegressor,49,-0.042219,"{'criterion': 'mse', 'max_depth': 5, 'n_estima...",0.015805


Do an evaluation summary of the best performing model

Define this as a classification problem and get a best performing classification model using grid search and cross-validation (this can take some time)